In [3]:
import xarray as xr
import numpy as np
import pandas as pd
import sys

def open_datasets(year, variable, months, way):
    datasets = []
    for month in months:
        filename = f'ERA5_{year}-{month}_{variable}.nc'
        dataset = xr.open_dataset(f'{way}{variable}/{filename}')
        datasets.append(dataset)
    return xr.concat(datasets, dim='time')

def open_and_concatenate(year, variable, months, way, level=None):
    if variable == 'geopotential':
        datasets = [xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc') for month in months]
        datasets = [dataset.sel(level=level) for dataset in datasets]
        return xr.concat(datasets, dim='time')
    else:
        datasets = [xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc') for month in months]
        return xr.concat(datasets, dim='time')

def process_storm_data(dates, year, dew_point_xr, specific_var):
    index_start_october = dates[(dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year + 1)].index[-1]

    for i in range(index_start_october, index_end_march + 1):
        start_date = dates.at[i, 'start_date']
        end_date = dates.at[i, 'end_date']
        new_dataset = dew_point_xr[specific_var].sel(time=slice(start_date, end_date))
        yield new_dataset

# Define a function to calculate statistics
def calculate_statistics(data_array):
    return {
        'mean': np.mean(data_array),
        'min': np.min(data_array),
        'max': np.max(data_array),
        'std': np.std(data_array),
        'skew': pd.Series(data_array.reshape(-1)).skew(),
        'kurtosis': pd.Series(data_array.reshape(-1)).kurtosis()
    }

def main(folder, year):
    year = int(year)
    year_next = year + 1
    if year_next == 2003:
        year_next = 2005
    month_act = [10, 11, 12]
    month_next = [1, 2, 3]
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

    if year in [1990, 2021]:
        months = month_act + month_next if year == 1990 else month_next + month_act
        dew_point_xr = open_datasets(str(year), folder, months, way)
    else:
        dew_point_xr_act = open_datasets(str(year), folder, month_act, way)
        dew_point_xr_next = open_datasets(str(year_next), folder, month_next, way)
        dew_point_xr = xr.concat([dew_point_xr_act, dew_point_xr_next], dim='time')

    specific_var = next(var for var in dew_point_xr.variables if var not in ['longitude', 'latitude', 'time'])
    dates = pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
    for new_dataset in process_storm_data(dates, year, dew_point_xr, specific_var):
        # Process each storm dataset here
        pass

'''if __name__ == '__main__':
    folder = sys.argv[1]
    year = sys.argv[2]
    main(folder, year)'''

"if __name__ == '__main__':\n    folder = sys.argv[1]\n    year = sys.argv[2]\n    main(folder, year)"

In [ ]:
# open a dataset
year = str(1990)

months = [1, 2, 3, 10 ,11, 12]

folder = '2m_dewpoint_temperature'

way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

test = open_datasets(year, folder, months, way)
test

In [ ]:
specific_var = next(var for var in test.variables if var not in ['longitude', 'latitude', 'time'])
dates = pd.read_csv('/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv')

test_process = process_storm_data(dates, year, test, 'd2m')

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import sys
from datetime import datetime

# Define a function to open datasets and concatenate them
def open_and_concatenate(year, variable, months, way, level=None):
    if variable == 'geopotential':
        datasets = [xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc') for month in months]
        datasets = [dataset.sel(level=level) for dataset in datasets]
        return xr.concat(datasets, dim='time')
    else:
        datasets = [xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc') for month in months]
        return xr.concat(datasets, dim='time')

# Define a function to calculate statistics
def calculate_statistics(data_array):
    return {
        'mean': np.mean(data_array),
        'min': np.min(data_array),
        'max': np.max(data_array),
        'std': np.std(data_array),
        'skew': pd.Series(data_array.reshape(-1)).skew(),
        'kurtosis': pd.Series(data_array.reshape(-1)).kurtosis()
    }

# Function to log processing details
def log_processing(variable, year):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f'Processed variable: {variable}, Year: {year}, Timestamp: {timestamp}'
    with open(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/{variable}/processing_log.txt', 'a') as log_file:
        log_file.write(log_message + '\n')

# Main function to process data
def process_data(variable, year):
    year = int(year)
    year_next = year + 1
    month_act = [10, 11, 12]
    month_next = [1, 2, 3]
    if variable == 'geopotential':
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/PL/'
    else:
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

    # Open and concatenate datasets
    if variable == 'geopotential':
        level = 500
        if year in [1990, 2021]:
            months = month_act + month_next if year == 1990 else month_next + month_act
            dataset = open_and_concatenate(str(year), variable, months, way, level)
        else:
            dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
            dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
            dataset = xr.concat([dataset_act, dataset_next], dim='time')
    else:
        if year in [1990, 2021]:
            months = month_act + month_next if year == 1990 else month_next + month_act
            dataset = open_and_concatenate(str(year), variable, months, way)
        else:
            dataset_act = open_and_concatenate(str(year), variable, month_act, way)
            dataset_next = open_and_concatenate(str(year_next), variable, month_next, way)
            dataset = xr.concat([dataset_act, dataset_next], dim='time')

    # Determine the specific variable to extract
    specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time'])

    # Import all tracks and convert dates
    dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
    dates['year'] = dates['start_date'].dt.year

    # Find the indices for storms within the specified timeframe
    if year == 1990:
        index_start_march = dates[(dates['start_date'].dt.month >= 10) & (dates['year'] == year)].index[0]
        index_end_october = dates[(dates['end_date'].dt.month <= 3) & (dates['year'] == year)].index[0]
    else:
        index_start_october = dates[(dates['start_date'].dt.month >= 10) & (dates['year'] == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['year'] == year_next)].index[-1]

    # Process each storm
    for i in range(index_start_october, index_end_march + 1):
        track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_1_hour/tc_irad_{i+1}_interp.txt')
        start_date = dates.at[i, 'start_date']
        end_date = dates.at[i, 'end_date']
        storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

        # Initialize lists to store statistics
        stats = {'mean': [], 'min': [], 'max': [], 'std': [], 'skew': [], 'kurtosis': []}

        # Calculate statistics for each time step
        for time_step in storm_data.time:
            data_slice = storm_data.sel(time=time_step).values
            step_stats = calculate_statistics(data_slice)
            for key in stats:
                stats[key].append(step_stats[key])

        # Save statistics to CSV files
        for key in stats:
            pd.DataFrame(stats[key]).to_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/{variable}/storm_{i+1}/{key}_{i+1}.csv')

    # Log the processing details
    log_processing(variable, year)

KeyboardInterrupt: 

In [26]:
# tests for 1990
variable = '10m_u_component_of_wind'
year = 2002
year_next = year + 1
month_act = [10, 11, 12]
month_next = [1, 2, 3]

if variable == 'geopotential':
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/PL/'
else:
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

# Open and concatenate datasets
if variable == 'geopotential':
    level = 500
    if year == 1990:
        dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
    elif year == 2021:
        dataset = open_and_concatenate(str(year), variable, month_next, way, level)
    else:
        dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
else:
    if year == 1990:
        dataset_act = open_and_concatenate(str(year), variable, month_next, way)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
    elif year == 2021:
        dataset = open_and_concatenate(str(year), variable, month_next, way)
    else:
        dataset_act = open_and_concatenate(str(year), variable, month_act, way)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')

# Determine the specific variable to extract
specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time','level'])

# Import all tracks and convert dates
dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
dates['year'] = dates['start_date'].dt.year

# Find the indices for storms within the specified timeframe
if year == 1990:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
elif year == 2021:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
else:
    #index_start_october = dates[(dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)].index[0]
    #if year_next == 2003: 
        #index_start_october = dates[(dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)].index[0]
        #index_end_march = index_start_october
    #else:
    index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month <= 3))].index[0]
    index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
    index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
    if len(index_end_march_first) > 0:
        index_end_march = index_end_march_first[0]
    else:
        if year_next == 2003:
            year_next = 2005
            index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
        index_end_march = index_end_march_second[0] - 1
# Process each storm
'''for i in range(index_start_october, index_end_march + 1):
    track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_1_hour/tc_irad_{i+1}_interp.txt')
    start_date = dates.at[i, 'start_date']
    end_date = dates.at[i, 'end_date']
    storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

    # Initialize lists to store statistics
    stats = {'mean': [], 'min': [], 'max': [], 'std': []}
    #, 'skew': [], 'kurtosis': []'''

'''# Calculate statistics for each time step
    for time_step in storm_data.time:
        data_slice = storm_data.sel(time=time_step).values
        step_stats = calculate_statistics(data_slice)
        for key in stats:
            stats[key].append(step_stats[key])

    # Save statistics to CSV files
    for key in stats:
        pd.DataFrame(stats[key]).to_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/{variable}/storm_{i+1}/{key}_{i+1}.csv')'''

IndexError: index 0 is out of bounds for axis 0 with size 0

In [28]:
index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month <= 3))].index[0]
index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
if len(index_end_march_first) > 0:
    index_end_march = index_end_march_first[0]
else:
    if year_next == 2003:
        year_next = 2005
        index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
    index_end_march = index_end_march_second[0] - 1
#test_index = [(dates['end_date'].dt.month <= 10)].index[0]
#[((dates['end_date'].dt.year == year) & [(dates['end_date'].dt.month <= 10))].index[0]

In [23]:
index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index

if len(index_end_march_first) > 0:
    index_end_march = index_end_march_first[0]
else:
    index_end_march = index_end_march_second[0] - 1

In [19]:
test = xr.open_dataset('/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/PL/geopotential/ERA5_1990-1_geopotential.nc')
#select level 500 of the geopotential

level = 500

test = test.sel(level=level)
test

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 744)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    level      int32 500
  * time       (time) datetime64[ns] 1990-01-01 ... 1990-01-31T23:00:00
Data variables:
    z          (time, latitude, longitude) float32 ...
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-06 17:12:07 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [20]:
test = test['z'].sel(time=slice(start_date, end_date))
test

<xarray.DataArray 'z' (time: 184, latitude: 721, longitude: 1440)>
[191036160 values with dtype=float32]
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    level      int32 500
  * time       (time) datetime64[ns] 1990-01-21T03:00:00 ... 1990-01-28T18:00:00
Attributes:
    units:          m**2 s**-2
    long_name:      Geopotential
    standard_name:  geopotential

In [22]:
dataset.sel(time=slice(start_date, end_date))

<xarray.Dataset>
Dimensions:    (longitude: 1440, latitude: 721, time: 4368)
Coordinates:
  * longitude  (longitude) float32 0.0 0.25 0.5 0.75 ... 359.0 359.2 359.5 359.8
  * latitude   (latitude) float32 90.0 89.75 89.5 89.25 ... -89.5 -89.75 -90.0
    level      int32 500
  * time       (time) datetime64[ns] 1990-10-01 ... 1990-03-31T23:00:00
Data variables:
    z          (time, latitude, longitude) float32 5.097e+04 ... 4.836e+04
Attributes:
    Conventions:  CF-1.6
    history:      2024-05-06 16:38:56 GMT by grib_to_netcdf-2.28.1: /opt/ecmw...

In [ ]:
year = 1990
year_next = 1991
#index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
#index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]

if year == 1990:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]-1
else:
    index_start_october = dates[(dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]

In [ ]:
sys.argv[1]='2m_dewpoint_temperature'
sys.argv[2]='1990'

if __name__ == '__main__':
    variable = sys.argv[1]
    year = sys.argv[2]
    process_data(variable, year)

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import sys
from datetime import datetime
# might be a problem for ML data, since the data is quite large in size
# Define a function to open datasets and concatenate them
def open_and_concatenate(year, variable, months, way, level=0):
    datasets = [xr.open_dataset(f'{way}{variable}/ERA5_{year}-{month}_{variable}.nc') for month in months]
    
    if variable == 'geopential' and level != 0:
        datasets = [dataset.sel(level=level) for dataset in datasets]
    
    return xr.concat(datasets, dim='time')

# Define a function to calculate statistics
def calculate_statistics(data_array):
    return {
        'mean': np.mean(data_array),
        'min': np.min(data_array),
        'max': np.max(data_array),
        'std': np.std(data_array),
    }

# Function to log processing details
def log_processing(variable, year, level, storm_number):
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    log_message = f'Processed variable: {variable}, Year: {year}, Level: {level}, Timestamp: {timestamp}, Storm number:{storm_number}'
    with open(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/processing_log.txt', 'a') as log_file:
        log_file.write(log_message + '\n')

# Main function to process data
def process_data(variable, year, level=0):
    year = int(year)
    year_next = year + 1
    month_act = [10, 11, 12]
    month_next = [1, 2, 3]
    if variable == 'geopotential':
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/PL/'
    else:
        way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

    # Open and concatenate datasets
    if year == 1990:
        dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')
    elif year == 2021:
        dataset = open_and_concatenate(str(year), variable, month_next, way, level)
    else:
        dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
        dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
        dataset = xr.concat([dataset_act, dataset_next], dim='time')

    # Determine the specific variable to extract
    specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

    # Import all tracks and convert dates
    dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
    dates['year'] = dates['start_date'].dt.year

    # Find the indices for storms within the specified timeframe
    if year == 1990:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
    elif year == 2021:
        index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
        index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
    else:
        index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month <= 3))].index[0]
        index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
        index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
        if len(index_end_march_first) > 0:
            index_end_march = index_end_march_first[0]
        else:
            if year_next == 2003:
                year_next = 2005
                index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
            index_end_march = index_end_march_second[0] - 1
    # Process each storm
    for i in range(index_start_october, index_end_march + 1):
        track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_1_hour/tc_irad_{i+1}_interp.txt')
        start_date = dates.at[i, 'start_date']
        end_date = dates.at[i, 'end_date']
        storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

        # Initialize lists to store statistics
        stats = {'mean': [], 'min': [], 'max': [], 'std': []}
        #, 'skewness': [], 'kurtosis': []

        # Calculate statistics for each time step
        for time_step in storm_data.time:
            data_slice = storm_data.sel(time=time_step).values
            step_stats = calculate_statistics(data_slice)
            for key in stats:
                stats[key].append(step_stats[key])

        # Save statistics to CSV files
        for key in stats:
            pd.DataFrame(stats[key]).to_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/{variable}/storm_{i+1}/{key}_{i+1}_{level}.csv')

    # Log the processing details
    log_processing(variable, year, level, i+1)

if __name__ == '__main__':
    variable = 'total_totals_index'
    year = 2017
    sys = 0
    level = sys
    process_data(variable, year, level)

IndexError: index 0 is out of bounds for axis 0 with size 0

In [5]:
variable = '10m_u_component_of_wind'
year = 2017
sys = 0
level = sys

year = int(year)
year_next = year + 1
month_act = [10, 11, 12]
month_next = [1, 2, 3]
if variable == 'geopotential':
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/PL/'
else:
    way = '/work/FAC/FGSE/IDYST/tbeucler/default/raw_data/ECMWF/ERA5/SL/'

# Open and concatenate datasets
if year == 1990:
    dataset_act = open_and_concatenate(str(year), variable, month_next, way, level)
    dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
    dataset = xr.concat([dataset_act, dataset_next], dim='time')
elif year == 2021:
    dataset = open_and_concatenate(str(year), variable, month_next, way, level)
else:
    dataset_act = open_and_concatenate(str(year), variable, month_act, way, level)
    dataset_next = open_and_concatenate(str(year_next), variable, month_next, way, level)
    dataset = xr.concat([dataset_act, dataset_next], dim='time')

# Determine the specific variable to extract
specific_var = next(var for var in dataset.variables if var not in ['longitude', 'latitude', 'time', 'level'])

# Import all tracks and convert dates
dates = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/storms_start_end.csv', parse_dates=['start_date', 'end_date'])
dates['year'] = dates['start_date'].dt.year

# Find the indices for storms within the specified timeframe
if year == 1990:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next)].index[0]
elif year == 2021:
    index_start_october = dates[(dates['start_date'].dt.month <= 3) & (dates['start_date'].dt.year == year)].index[0]
    index_end_march = dates[(dates['end_date'].dt.year == 2021)].index[0]
else:
    index_start_october = dates[((dates['start_date'].dt.month >= 10) & (dates['start_date'].dt.year == year)) | ((dates['start_date'].dt.year == year_next) & (dates['start_date'].dt.month <= 3))].index[0]
    index_end_march_first = dates[((dates['end_date'].dt.month <= 3) & (dates['end_date'].dt.year == year_next))].index
    index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
    if len(index_end_march_first) > 0:
        index_end_march = index_end_march_first[0]
    else:
        if year_next == 2003:
            year_next = 2005
            index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
        elif year_next == 2018:
            year_next = 2020
            index_end_march_second = dates[((dates['end_date'].dt.year == year_next) & (dates['end_date'].dt.month <= 12))].index
        index_end_march = index_end_march_second[0] - 1
# Process each storm
for i in range(index_start_october, index_end_march + 1):
    track = pd.read_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/tc_irad_tracks/tc_1_hour/tc_irad_{i+1}_interp.txt')
    start_date = dates.at[i, 'start_date']
    end_date = dates.at[i, 'end_date']
    storm_data = dataset[specific_var].sel(time=slice(start_date, end_date))

    # Initialize lists to store statistics
    stats = {'mean': [], 'min': [], 'max': [], 'std': []}
    #, 'skewness': [], 'kurtosis': []

    # Calculate statistics for each time step
    for time_step in storm_data.time:
        data_slice = storm_data.sel(time=time_step).values
        step_stats = calculate_statistics(data_slice)
        for key in stats:
            stats[key].append(step_stats[key])

    # Save statistics to CSV files
    for key in stats:
        pd.DataFrame(stats[key]).to_csv(f'/work/FAC/FGSE/IDYST/tbeucler/default/fabien/repos/curnagl/datasets/{variable}/storm_{i+1}/{key}_{i+1}_{level}.csv')